In [1]:
import time
import tqdm 
import pandas as pd 

from __init__ import * 
from models.utils import * 

import torch
import torch.optim as optim 

from models.lstm import * 

from transformers import BertTokenizer

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class args:
    seed = 42
    max_seq_length = 256
    batch_size = 512
    hidden_dim = 128
    num_epochs = 10
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
train_path = os.path.join(YELP_DIR, 'train.csv')
valid_path = os.path.join(YELP_DIR, 'valid.csv')

train = pd.read_csv(train_path, encoding='utf-8-sig')
valid = pd.read_csv(valid_path, encoding='utf-8-sig')

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
train.loc[:, 'stars'] = train.loc[:, 'stars'].apply(sentiment_score)
valid.loc[:, 'stars'] = valid.loc[:, 'stars'].apply(sentiment_score)

train = train.dropna(axis=0)
train = valid.dropna(axis=0)

In [18]:
train_dataloader = get_dataloader(args, train, tokenizer)
valid_dataloader = get_dataloader(args, valid, tokenizer)

In [19]:
args.vocab_size = tokenizer.vocab_size
args.embedding_dim = 512
args.output_dim = 1
args.n_layers = 2 
args.bidirectional = True 
args.dr_rate = 0 
args.learning_rate = 1e-3

models = LSTMClassifier(args.vocab_size, args.embedding_dim, args.hidden_dim, args.output_dim, args.n_layers, args.bidirectional, args.dr_rate).to(args.device)

optimizer = optim.Adam(models.parameters(), lr=args.learning_rate)
criterion = nn.BCEWithLogitsLoss().to(args.device)

In [20]:
results = lstm_train(args, models, train_dataloader, valid_dataloader, optimizer, criterion)

Evaluating..: 100%|██████████| 468/468 [01:32<00:00,  5.08it/s]


epoch [1/10] | elapsed time: 3m, 2.71s
train loss: 0.000003	train accuracy: 99.96%
valid loss: 0.000000	valid accuracy: 100.00% 



AttributeError: type object 'args' has no attribute 'save_parameters'